# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 11:16AM,256917,10,0086363970,ISDIN Corporation,Released
1,Feb 8 2023 11:16AM,256917,10,0086363971,ISDIN Corporation,Released
2,Feb 8 2023 11:16AM,256917,10,0086363974,ISDIN Corporation,Released
3,Feb 8 2023 11:16AM,256917,10,0086363973,ISDIN Corporation,Released
4,Feb 8 2023 11:16AM,256917,10,0086363985,ISDIN Corporation,Released
5,Feb 8 2023 11:16AM,256917,10,0086363987,ISDIN Corporation,Released
6,Feb 8 2023 11:16AM,256917,10,0086363986,ISDIN Corporation,Released
7,Feb 8 2023 10:59AM,256916,19,60019601,"GUSA Granules USA, Inc.",Released
8,Feb 8 2023 10:49AM,256915,19,2022102810,"GUSA Granules USA, Inc.",Released
9,Feb 8 2023 10:49AM,256915,19,2022102811,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,256912,Released,2
37,256914,Released,1
38,256915,Released,2
39,256916,Released,1
40,256917,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256912,NaN,NaN,2.0
256914,NaN,NaN,1.0
256915,NaN,NaN,2.0
256916,NaN,NaN,1.0
256917,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25.0,14.0,13.0
256824,0.0,0.0,1.0
256826,0.0,0.0,113.0
256827,0.0,12.0,4.0
256828,0.0,9.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25,14,13
256824,0,0,1
256826,0,0,113
256827,0,12,4
256828,0,9,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,14,13
1,256824,0,0,1
2,256826,0,0,113
3,256827,0,12,4
4,256828,0,9,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,14,13
1,256824,,,1
2,256826,,,113
3,256827,,12,4
4,256828,,9,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 11:16AM,256917,10,ISDIN Corporation
7,Feb 8 2023 10:59AM,256916,19,"GUSA Granules USA, Inc."
8,Feb 8 2023 10:49AM,256915,19,"GUSA Granules USA, Inc."
10,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc."
11,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation
13,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC"
14,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC"
16,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC"
17,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC"
21,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 11:16AM,256917,10,ISDIN Corporation,,,7
1,Feb 8 2023 10:59AM,256916,19,"GUSA Granules USA, Inc.",,,1
2,Feb 8 2023 10:49AM,256915,19,"GUSA Granules USA, Inc.",,,2
3,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",,,1
4,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,,,2
5,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",,,1
6,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",,,2
7,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",,,1
8,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC",,,4
9,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 11:16AM,256917,10,ISDIN Corporation,7,,
1,Feb 8 2023 10:59AM,256916,19,"GUSA Granules USA, Inc.",1,,
2,Feb 8 2023 10:49AM,256915,19,"GUSA Granules USA, Inc.",2,,
3,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1,,
4,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,,
5,Feb 8 2023 10:24AM,256911,16,"SHL Pharma, LLC",1,,
6,Feb 8 2023 10:15AM,256910,10,"Snap Medical Industries, LLC",2,,
7,Feb 8 2023 10:12AM,256909,10,"Virtue Rx, LLC",1,,
8,Feb 8 2023 10:11AM,256908,10,"ClearSpec, LLC",4,,
9,Feb 8 2023 10:09AM,256907,10,"Senseonics, Incorporated",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 11:16AM,256917,10,ISDIN Corporation,7,,
1,Feb 8 2023 10:59AM,256916,19,"GUSA Granules USA, Inc.",1,,
2,Feb 8 2023 10:49AM,256915,19,"GUSA Granules USA, Inc.",2,,
3,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1,,
4,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 11:16AM,256917,10,ISDIN Corporation,7,NaN,NaN
1,Feb 8 2023 10:59AM,256916,19,"GUSA Granules USA, Inc.",1,NaN,NaN
2,Feb 8 2023 10:49AM,256915,19,"GUSA Granules USA, Inc.",2,NaN,NaN
3,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1,NaN,NaN
4,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 11:16AM,256917,10,ISDIN Corporation,7,0.0,0.0
1,Feb 8 2023 10:59AM,256916,19,"GUSA Granules USA, Inc.",1,0.0,0.0
2,Feb 8 2023 10:49AM,256915,19,"GUSA Granules USA, Inc.",2,0.0,0.0
3,Feb 8 2023 10:30AM,256914,19,"GUSA Granules USA, Inc.",1,0.0,0.0
4,Feb 8 2023 10:24AM,256912,10,ISDIN Corporation,2,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4367227,119,38.0,0.0
15,1284240,166,40.0,25.0
16,513784,2,0.0,0.0
19,2055020,25,25.0,0.0
21,256870,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4367227,119,38.0,0.0
1,15,1284240,166,40.0,25.0
2,16,513784,2,0.0,0.0
3,19,2055020,25,25.0,0.0
4,21,256870,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119,38.0,0.0
1,15,166,40.0,25.0
2,16,2,0.0,0.0
3,19,25,25.0,0.0
4,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,15,Released,166.0
2,16,Released,2.0
3,19,Released,25.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,25.0,0.0,0.0,0.0
Executing,38.0,40.0,0.0,25.0,0.0
Released,119.0,166.0,2.0,25.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,25.0,0.0,0.0,0.0
1,Executing,38.0,40.0,0.0,25.0,0.0
2,Released,119.0,166.0,2.0,25.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,25.0,0.0,0.0,0.0
1,Executing,38.0,40.0,0.0,25.0,0.0
2,Released,119.0,166.0,2.0,25.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()